In [1]:
import numpy as np
import pandas as pd

In [2]:
# Compara dois estados e computa a diferença
def h(state_begin, state_end):
    cost = 0
    # Percorre a matriz para calcular o custo relacionado a cada elemento
    for index, v in np.ndenumerate(state_end):
        if (v != ''):
            cost += abs(index[0] - np.where(state_begin == v)[0][0])  # Calcula a distância horizontal 
            cost += abs(index[1] - np.where(state_begin == v)[1][0])  # Calcula a distância vertical
    
    return cost

# Modifica a posição do quadrado vazio no tabuleiro
def move(direction, state_tmp, state_end, pos_x, pos_y, cost_g, node_id):
    # Movimenta o espaço vazio para a esquerda
    if (direction == 'left'):
        a, b = state_tmp[pos_x][pos_y-1], state_tmp[pos_x][pos_y] 
        state_tmp[pos_x][pos_y-1], state_tmp[pos_x][pos_y] = b, a
    # Movimenta o espaço vazio para a direita
    elif (direction == 'right'):
        a, b = state_tmp[pos_x][pos_y], state_tmp[pos_x][pos_y+1] 
        state_tmp[pos_x][pos_y], state_tmp[pos_x][pos_y+1] = b, a
    # Movimenta o espaço vazio para baixo
    elif (direction == 'down'):
        a, b = state_tmp[pos_x][pos_y], state_tmp[pos_x+1][pos_y] 
        state_tmp[pos_x][pos_y], state_tmp[pos_x+1][pos_y] = b, a
    # Movimenta o espaço vazio para a cima
    elif (direction == 'up'):
        a, b = state_tmp[pos_x-1][pos_y], state_tmp[pos_x][pos_y] 
        state_tmp[pos_x-1][pos_y], state_tmp[pos_x][pos_y] = b, a
        
    # Calcula o custo da nova configuração
    cost_h = h(state_tmp, state_end)
    cost_f = cost_g + cost_h

    # Devolve o novo estado 
    new_row = {
      "Node": node_id,
      "State": state_tmp,
      "Action": direction,
      "Cost_g": cost_g,
      "Cost_h": cost_h,  
      "Cost_f": cost_f
    }
    
    return new_row

In [3]:
# Estado inicial do jogo
state = np.array([['1', '8', '2'], 
                  [ '', '4', '3'], 
                  ['7', '6', '5']])

# Estado final desejado
state_end = np.array([['1', '2', '3'], 
                      ['4', '5', '6'], 
                      ['7', '8', '']])

# Dicionário que representa um estado na fronteira
data = {
      "Node": [0],
      "State": [state],
      "Action": [''],
      "Cost_g": [0],
      "Cost_h": [h(state, state_end)],
      "Cost_f": [h(state, state_end)]
    }

# Coloca o estado inicial na fronteira
boundary_df = pd.DataFrame(data=data)
print('## Iteraction 0')
print(boundary_df)

node_id = 0
iteraction = 1
stop = False
# Percorre a árvore de estados até atingir o estado final
while (iteraction < 15):
    print('## Iteraction ' + str(iteraction))
    
    # Identifica qual é o nó de menor custo
    cheapestNode = boundary_df.loc[boundary_df['Cost_f'] == boundary_df['Cost_f'].min()].index[0]
    cheapestNodeId = boundary_df.at[cheapestNode, 'Node']
    
    # Identifica qual foi o movimento que gerou esse nó, para não voltar para o estado pai
    lastMove = boundary_df.at[cheapestNode, 'Action']
    
    # Identifica a configuração de tabuleiro do nó de menor custo
    state = boundary_df.at[cheapestNode, 'State']
    state = np.array(state)
    
    # Identifica o g() do nó atual e já calcula o valor que será utilizado nos filhos
    cost_g = boundary_df.at[cheapestNode, 'Cost_g']
    cost_g += 1
    
    # Pega a posição horizontal e vertical do quadrado vazio
    pos_x = np.where(state == '')[0][0]
    pos_y = np.where(state == '')[1][0]
    
    # Gera todos os filhos possíveis a partir da posição atual
    # MOVENDO PARA A ESQUERDA
    if ((not stop) and (pos_y > 0) and (lastMove != 'right')): 
        node_id += 1
        state_tmp = np.array(state)
        new_row = move('left', state_tmp, state_end, pos_x, pos_y, cost_g, node_id)
        boundary_df = boundary_df.append(new_row, ignore_index=True)
        
        # Se encontrar o estado final, encerra o programa
        state_tmp = np.array(new_row['State'])
        if np.array_equal(state_tmp, state_end):
            stop = True
        
    # MOVENDO PARA A DIREITA    
    if ((not stop) and (pos_y < state.shape[0] - 1) and (lastMove != 'left')): 
        node_id += 1
        state_tmp = np.array(state)
        new_row = move('right', state_tmp, state_end, pos_x, pos_y, cost_g, node_id)
        boundary_df = boundary_df.append(new_row, ignore_index=True)
        
        # Se encontrar o estado final, encerra o programa
        state_tmp = np.array(new_row['State'])
        if np.array_equal(state_tmp, state_end):
            stop = True
    
    # MOVENDO PARA BAIXO
    if ((not stop) and (pos_x < state.shape[1] - 1) and (lastMove != 'up')):   
        node_id += 1
        state_tmp = np.array(state)
        new_row = move('down', state_tmp, state_end, pos_x, pos_y, cost_g, node_id)
        boundary_df = boundary_df.append(new_row, ignore_index=True)
        
        # Se encontrar o estado final, encerra o programa
        state_tmp = np.array(new_row['State'])
        if np.array_equal(state_tmp, state_end):
            stop = True
    
    # MOVENDO PARA CIMA
    if ((not stop) and (pos_x > 0) and (lastMove != 'down')):     
        node_id += 1
        state_tmp = np.array(state)
        new_row = move('up', state_tmp, state_end, pos_x, pos_y, cost_g, node_id)
        boundary_df = boundary_df.append(new_row, ignore_index=True)
        
        # Se encontrar o estado final, encerra o programa
        state_tmp = np.array(new_row['State'])
        if np.array_equal(state_tmp, state_end):
            stop = True
    
    # Verifica se algum dos filhos já existe na fronteira
    
    # Remove o pai da fronteira
    cheapestNode = boundary_df.loc[boundary_df['Node'] == cheapestNodeId].index[0]
    boundary_df.drop(index=cheapestNode, inplace=True)
    
    # Ordena a fronteira pelo custo
    boundary_df.sort_values(by=['Cost_f', 'Cost_h', 'Cost_g', 'Node'], inplace=True)
    print(boundary_df)
    
    # Verifica se a condição de saída já foi alcançada
    if (stop):
        break    
    iteraction += 1
    
print('Fim')

## Iteraction 0
   Node                             State Action  Cost_g  Cost_h  Cost_f
0     0  [[1, 8, 2], [, 4, 3], [7, 6, 5]]              0       9       9
## Iteraction 1
   Node                             State Action  Cost_g  Cost_h  Cost_f
1     1  [[1, 8, 2], [4, , 3], [7, 6, 5]]  right       1       8       9
2     2  [[1, 8, 2], [7, 4, 3], [, 6, 5]]   down       1      10      11
3     3  [[, 8, 2], [1, 4, 3], [7, 6, 5]]     up       1      10      11
## Iteraction 2
   Node                             State Action  Cost_g  Cost_h  Cost_f
4     5  [[1, 8, 2], [4, 6, 3], [7, , 5]]   down       2       7       9
5     6  [[1, , 2], [4, 8, 3], [7, 6, 5]]     up       2       7       9
3     4  [[1, 8, 2], [4, 3, ], [7, 6, 5]]  right       2       9      11
1     2  [[1, 8, 2], [7, 4, 3], [, 6, 5]]   down       1      10      11
2     3  [[, 8, 2], [1, 4, 3], [7, 6, 5]]     up       1      10      11
## Iteraction 3
   Node                             State Action  Cost_g  Co